# Testing notebook

This notebook is meant for visualizing stuff and testing code. 

In [1]:
import numpy as np
import pandas as pd
import os
import warnings
from dataclasses import dataclass
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from contextlib import contextmanager
from typing import Generator
import demoji
warnings.filterwarnings("ignore", category=FutureWarning)

import spacy
from spacy.tokens import Doc, Span, Token
from spacy import displacy
from load_spacy import nlp

In [30]:

def load_data(data_dir:str) -> pd.DataFrame:
    """Loads a .jsonl file"""
    return pd.read_json(data_dir, lines=True)
    

test_docs = [
    "It was Jane's car that got stolen last night.",
    "It is my boss who will send me home today.",
    "It is me who tom saw today.",
    "It was your friend Suzy I met!",
    "Was it the boy in the hat who saw you?",
    "It wasn't the dog in the tree that ate the apple."
]



In [126]:
s = "it was Jane’s car that got stolen last night"
doc = nlp(s)


def has_children(token) -> bool:
    return token.n_lefts + token.n_rights > 0
        
def parse_to_string(token):
    stack = [(token, False)]
    result = []

    while stack:
        current_token, processed = stack.pop()

        if not processed:
            stack.append((current_token, True))

            if current_token.n_lefts + current_token.n_rights > 0:
                result.append('(' + current_token.text)

                for child in reversed(list(current_token.children)):
                    stack.append((child, False))
            else:
                result.append('(' + current_token.text + ')')
        else:
            result.append(')')

    return ' '.join(result)

def convert_dependency_tree(sentence):
    result = ""
    stack = [(sentence.root, 0)]
    
    while stack:
        token, level = stack.pop()
        result += " " * level + "(" + token.text + " "
        
        children = [child for child in token.children][::-1]
        if children:
            stack.extend([(child, level + 1) for child in children])
        else:
            result += ")"
    
    return result.strip()
    
    
    
    

def generate_dependency_string(sentence:Span):
    return convert_dependency_tree(sentence)

for sent in doc.sents:
    print(generate_dependency_string(sent))


target1 = "(was (it) (car (Jane  (‘s) (stolen (that) (got) (night (last)))))"
target2 = "(was-be-VBD-Root (it-it-PRP-SUBJ) (car- (Jane  (‘s) (stolen (that) (got) (night (last)))))"

(was  (it ) (car   (Jane    (’s )  (stolen    (that )   (got )   (night     (last )


In [97]:
REGISTERD_FEATURES = {}

class Feature:
    
    def __init__(self, func):
        self.func = func    
        self.name = func.__name__
        
    def __call__(self, doc):
        features = self.func(doc)
        return self.add_one_to_func_output()
    
    def add_one_to_func_output(self):
        return self.features + 1
    
    @classmethod
    def register(cls, func):
        func = cls(func)
        REGISTERD_FEATURES[func.name] = func
        return func

 
@Feature.register
def func1(doc:int):
    return doc

@Feature.register
def func2(doc:int):
    return doc


SyntaxError: invalid syntax (3991570972.py, line 14)

In [96]:
REGISTERD_FEATURES["func1"].features





1